# Nanodegree Engenheiro de Machine Learning
## Projeto final
Gabriel Yan Mitoso

20 de Outubro de 2018

## Explorando os Dados
Nesta seção, vamos explorar e analisar os dados através de visualizações e códigos.

## Visualização exploratória
Nesta seção, vamos gerar diferentes histogramas e analisá-los.

## Pré-processamento de dados
Nesta seção, vamos identificar e analisar possíveis outliers, assim como transformar dados não numéricos para facilitar o processamento.

## Implementação
Nesta seção, vamos implementar os algoritmos de K-Nearest Neighbors e Regressão Logística, utilizando Log Loss como métrica.

## Refinamento
Nesta seção, vamos otimizar os algoritmos de K-Nearest Neighbors e Regressão Logística.